In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

#load the file and inspect the first few rows
filepath = r"C:\Users\Admin\Machine Learning Fundamentals\CSV_files\MachineLearning-Data.xlsx"
data = pd.read_excel(filepath)

#drop the empty column
data = data.drop(columns=['DA concentration'])
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   DA Concentration (uM)  50 non-null     int64  
 1   AA Concentration (uM)  50 non-null     int64  
 2   Peak Current (uA)      50 non-null     float64
 3   Peak Potential (mV)    50 non-null     float64
 4   Peak Area (uC)         50 non-null     float64
dtypes: float64(3), int64(2)
memory usage: 2.1 KB


,DA Concentration (uM),AA Concentration (uM),Peak Current (uA),Peak Potential (mV),Peak Area (uC)
0,0,100,8.763,133.447,40.418
1,0,150,9.755,147.679,44.194
2,0,200,14.559,172.276,70.734
3,0,300,21.614,174.853,112.173
4,0,400,31.215,164.517,168.123


In [3]:
#split the data into features and target
X = data.drop(columns=['DA Concentration (uM)','AA Concentration (uM)'])
y = data[['DA Concentration (uM)', 'AA Concentration (uM)']]

X.head()

,Peak Current (uA),Peak Potential (mV),Peak Area (uC)
0,8.763,133.447,40.418
1,9.755,147.679,44.194
2,14.559,172.276,70.734
3,21.614,174.853,112.173
4,31.215,164.517,168.123


In [4]:
y.head()

,DA Concentration (uM),AA Concentration (uM)
0,0,100
1,0,150
2,0,200
3,0,300
4,0,400


In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standarize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)